In [27]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import  ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import joblib
import warnings
warnings.filterwarnings('ignore')

In [28]:
df = pd.read_csv('Price.csv')
df.head()

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1850000000,61666.67
1,60,1,True,True,True,Shahran,1850000000,61666.67
2,79,2,True,True,True,Pardis,550000000,18333.33
3,95,2,True,True,True,Shahrake Qods,902500000,30083.33
4,123,2,True,True,True,Shahrake Gharb,7000000000,233333.33


In [29]:
df.sample(5)

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
145,70,2,False,True,True,Saadat Abad,1800000000,60000.00
40,58,1,True,True,True,Saadat Abad,3400000000,113333.33
31,49,1,True,True,True,West Ferdows Boulevard,2200000000,73333.33
92,75,2,True,True,True,Pirouzi,2300000000,76666.67
74,63,1,True,True,True,Sattarkhan,2150000000,71666.67


In [30]:
type(df)

pandas.core.frame.DataFrame

In [31]:
df.columns

Index(['Area', 'Room', 'Parking', 'Warehouse', 'Elevator', 'Address', 'Price',
       'Price(USD)'],
      dtype='object')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Area        150 non-null    int64  
 1   Room        150 non-null    int64  
 2   Parking     150 non-null    bool   
 3   Warehouse   150 non-null    bool   
 4   Elevator    150 non-null    bool   
 5   Address     150 non-null    object 
 6   Price       150 non-null    int64  
 7   Price(USD)  150 non-null    float64
dtypes: bool(3), float64(1), int64(3), object(1)
memory usage: 6.4+ KB


In [33]:
df.describe()

,Area,Room,Price,Price(USD)
count,150.000000,150.000000,1.500000e+02,150.000000
mean,92.600000,1.946667,3.163967e+09,105465.577467
std,36.849048,0.702695,3.109839e+09,103661.311760
min,40.000000,0.000000,3.600000e+06,120.000000
25%,66.250000,2.000000,1.285000e+09,42833.335000
50%,85.000000,2.000000,2.200000e+09,73333.330000
75%,113.750000,2.000000,3.795000e+09,126500.000000
max,300.000000,3.000000,2.000000e+10,666666.670000


In [34]:
df["Area"] = pd.to_numeric(df["Area"] , errors='coerce')

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Area        150 non-null    int64  
 1   Room        150 non-null    int64  
 2   Parking     150 non-null    bool   
 3   Warehouse   150 non-null    bool   
 4   Elevator    150 non-null    bool   
 5   Address     150 non-null    object 
 6   Price       150 non-null    int64  
 7   Price(USD)  150 non-null    float64
dtypes: bool(3), float64(1), int64(3), object(1)
memory usage: 6.4+ KB


In [36]:
df

,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1850000000,61666.67
1,60,1,True,True,True,Shahran,1850000000,61666.67
2,79,2,True,True,True,Pardis,550000000,18333.33
3,95,2,True,True,True,Shahrake Qods,902500000,30083.33
4,123,2,True,True,True,Shahrake Gharb,7000000000,233333.33
...,...,...,...,...,...,...,...,...
145,70,2,False,True,True,Saadat Abad,1800000000,60000.00
146,150,3,True,True,True,Golestan,20000000000,666666.67
147,95,2,True,True,True,Southern Janatabad,950000000,31666.67
148,127,2,True,True,True,Shahr-e-Ziba,6000000000,200000.00


In [37]:
df.shape

(150, 8)

In [38]:
print(df.dtypes)

Area            int64
Room            int64
Parking          bool
Warehouse        bool
Elevator         bool
Address        object
Price           int64
Price(USD)    float64
dtype: object


In [39]:
df.isnull().sum()

Area          0
Room          0
Parking       0
Warehouse     0
Elevator      0
Address       0
Price         0
Price(USD)    0
dtype: int64

In [40]:
df.dropna(inplace=True)

In [41]:
df.shape

(150, 8)

In [42]:
df['Parking'].value_counts(normalize=True)*100

True     82.666667
False    17.333333
Name: Parking, dtype: float64

In [43]:
pd.crosstab( df.Parking, df.Room )

Room,0,1,2,3
Parking,,,,
False,2,8,14,2
True,0,27,68,29


In [44]:
below_85 = df[ df.Area <= 85 ]
len(below_85)

77

In [45]:
df[ df.Area <= 85 ]["Parking"].value_counts( )

True     59
False    18
Name: Parking, dtype: int64